In [1]:
#import the required pacakge
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import LabelEncoder, RobustScaler

# importing ploting libraries
import matplotlib.pyplot as plt   


In [2]:
#to enable the plotting on Jupyter notebook
%matplotlib inline

In [3]:

## load data
prod_df = pd.read_csv('cproducts.csv')
tend_df = pd.read_csv('ctender.csv')



In [4]:
#check the shape of the data
print('prodf has {} rows and {}   columns '.format(prod_df.shape[0],prod_df.shape[1]))
print('tend_df has {} rows and {} column '.format(tend_df.shape[0],tend_df.shape[1]))

prodf has 11972 rows and 16   columns 
tend_df has 149752 rows and 13 column 


In [5]:
#just check the data types of the prod_df and tend_df
print ('Product data structure')
print prod_df.dtypes
print('*********************************************')
print('tend data structure')
print tend_df.dtypes

Product data structure
customerID                     object
DOB                            object
Gender                         object
State                          object
PinCode                       float64
transactionDate                object
store_code                      int64
store_description              object
till_no                         int64
transaction_number_by_till      int64
promo_code                     object
promotion_description          object
product_code                    int64
product_description            object
sale_price_after_promo        float64
discountUsed                   object
dtype: object
*********************************************
tend data structure
customerID                     object
DOB                            object
Gender                         object
State                          object
PinCode                       float64
transactionDate                object
store_code                      int64
store_description      

In [ ]:
#just will have glance of data 
prod_df.head(5)

In [ ]:
tend_df.head(5)

$ Few info about the data , just for the understanding of the data 

Future Group has taken the customers relation to next level by introducing the loyality program .
With this loyality program customer can accumulate point with online and offline partner using a single card .
Customers can accumulate points across Future Group formats(home town ,bog bazar,ezone,brand factory ,central etc.) , thereby making “shopping rewarding”.

 It is having many  loyality program 
 
1.Payback -PAYBACK is Europe's leading customer loyalty program, headquartered in Germany , with a total of 25.5 million active cardholders in Germany and Poland. In India, PAYBACK consolidated its position after taking a major stake in I-Mint. Today, with more than 30 partners with 1,500 outlets and 10 million card members, it’s the only program which works with market leading pan-India players from the banking, travel, petroleum and online sectors such as ICICI Bank, HPCL, Univercell, BookMyShow and MakeMyTrip.

Now that Future Group has become a part of this bandwagon, customers are bound to earn points at every step – from formats like Big Bazaar, Food Bazaar, Central, HomeTown, Ezone, Brand Factory and Future Bazaar.
These points can then be redeemed for air miles, movie tickets, air tickets and vice versa.

With Future Group and PAYBACK “Shopping is Rewarding”.


2.T24- will provide customers with a dual advantage all 24 hours of the day—‘Shop More, Talk More’ and ‘Talk More, Shop More’.
Customers will get shopping benefits for talking and talk-time benefits each time they shop.

3.Big Bazaar Profit Club- This is kind of membership program where user can pay 10000 and then shop 1000 each month for 12 months .Benefitting 2000 shoping in  a year.

Members of the Big Bazaar PROFIT CLUB will also receive the benefit of existing offers on their Payback cards and T24 mobile services.

4.Easyday Savings Club 

5.Future Pay mobile wallet 


Along with the loyality program  few festive offer also there time to time 

1. wednesday bazar
2. Mahabchat offer during 24-26 Jan
3. The Great Excahange Offer 

here data is availbale in two file product and tender.csv

$ Product.csv contains the list of user along with the product in which customer got discount for particular transaction

$ Tender.csv contain contains mainly about the mode of payment done for particular transaction like cash ,gift vouchers etc.

Points understood so far 

1. As i understood all the transaction listed in product.csv must present in tender.csv , will check it later 

2. Tender_Type and PaymentMode of tender.csv contains the same info its like redundant column 

3. As per the question "Predict the cluster Based on the store-location " so here store-code serves for store location so here instead  of store-location ,store-desription ,state ,Pin Code we can use only store-code rest of the thing is not much required what i feel

4.  Since Clustering should here consider only the behaviour of customer whch here can be identified with the mode of payment and type of discount used and how much they spent will be meaningful 

I am not confident enough in using promo_code,PromoDescription,Sale_price_after_Discount 

And  customerID, transactionDate,till No(billing counter number ),transaction_Number_by_till may not be significant 

So Final conclusion on importance of  variable based on my understanding will be  DOB (Age can be derived) ,Gender,Discount_used ,Mode Of payment ,Payment Amount can be significant 

 $ if all the product.csv is available in tender.csv ,later point i will merge



Start with basic thing and using the sample code which is available for this clustering

##Also can see NANA value In DOB which i will replace with nan value 


In [6]:
#replace all the NANA of DOB value to Null or Nan

prod_df['DOB']=prod_df['DOB'].replace('NANA', np.nan)


tend_df['DOB']=tend_df['DOB'].replace('NANA', np.nan)

In [7]:
# check if any missing number in both the data

prod_df.isnull().sum()


customerID                       0
DOB                            473
Gender                        1028
State                         1583
PinCode                        817
transactionDate                  0
store_code                       0
store_description                0
till_no                          0
transaction_number_by_till       0
promo_code                       0
promotion_description         8510
product_code                     0
product_description              0
sale_price_after_promo           0
discountUsed                     0
dtype: int64

In [8]:
#check if any missing number in present in the tender data 
#check the NAN value in the train data set 
tend_df.isnull().sum()


customerID                        0
DOB                            4630
Gender                         6663
State                         11650
PinCode                        5564
transactionDate                   0
store_code                        0
store_description                 0
till_no                           0
tender_type                       0
transaction_number_by_till        0
payment_amount_by_tender          0
PaymentUsed                    1223
dtype: int64

Observation from the above :
    #in product promotional description info is missing a lot hence may not contribute much in the model 
    #in both the data DOB,gender,State and pincode is missing but it is not less than 20 so will try to fill those value
    #both the data is having majority object type data type,may required to change for further processing 

In [ ]:
#check if product.csv is a subset of the other tender.csv return 1 if it is a subset of another else return list

def islistsubset(sublist,mainlist):
    for item in sublist:
     #   print item            
        if item  in mainlist:
            contains = 1
        else:
            print item
            contains = 0
       #     break;
    return contains

In [ ]:
#check with transaction number 
islistsubset(prod_df['transaction_number_by_till'].tolist(),tend_df['transaction_number_by_till'].tolist())

In [ ]:
#check with cutomerId
#tend_df['customerID']
islistsubset(prod_df['customerID'].tolist(),tend_df['customerID'].tolist())
#i='BBID_20482'
#if i in tend_df['customerID']:
#    print "true"

In [ ]:
#as we understood all the value of prod_df is present in tend_df 
#now we will try to merge both the data frame before that wanted if all the customerid of prod_df are unique (not repeting )
#check how many 
prod_df.customerID.unique().shape[0]


looks like few are repeting
check the repeting value

In [59]:
#just wanted to check which all are repeting
prod_df[prod_df.duplicated(['customerID'], keep=False)]

,customerID,DOB,Gender,State,PinCode,transactionDate,store_code,store_description,till_no,transaction_number_by_till,promo_code,promotion_description,product_code,product_description,sale_price_after_promo,discountUsed
15,BBID_204354,1993-03-05,male,NaN,999999.0,2016-10-01,4986,BB-MADURAI-KOCHADAI VILLAGE,8,98889,0001188959,"HO B1@15%,2@20%,4@25% OFF SEATING REG AZ",1000218054001,"GNG NP SLEEPER CHAIR 39x55x55, RED, NA",296.65,Payback
16,BBID_204354,1993-03-05,male,NaN,999999.0,2016-10-01,4986,BB-MADURAI-KOCHADAI VILLAGE,8,98888,NONPROMO,NaN,108004629,GH DAILY RAVA/SUJI 1Kg,46.00,Payback
31,BBID_204717,1948-10-23,female,MADHYA PRADESH,452001.0,2016-06-02,2906,BB-INDORE-TREASURE ISLAND,7,7465,NONPROMO,NaN,108100207,MANGO TOTAPURI,51.92,Payback
32,BBID_204717,1948-10-23,female,MADHYA PRADESH,452001.0,2017-06-01,2906,BB-INDORE-TREASURE ISLAND,8,57880,NONPROMO,NaN,108004590,SUNRICH SUNFLOWER OIL PP 1L,258.00,Payback
38,BBID_204891,1967-09-23,male,MADHYA PRADESH,452010.0,2015-11-03,2655,BB-INDORE-MALHAR MEGA MALL,4,18925,NONPROMO,NaN,108032496,SAFAL PEAS GREEN PEAS 200G,45.00,Payback
39,BBID_204891,1967-09-23,male,MADHYA PRADESH,452010.0,2015-11-03,2655,BB-INDORE-MALHAR MEGA MALL,4,18925,0001041645,"AZ 1@10,2@30 RS OFF Frozen Veg Snacks FB",300658528,MCCAIN FROZ VEG SNACK SMILES 175g,50.00,Payback
40,BBID_204900,1981-05-21,female,MADHYA PRADESH,452014.0,2016-10-24,2906,BB-INDORE-TREASURE ISLAND,1,20335,NONPROMO,NaN,300776411,BB-CB-27X30X208SWG NEW,14.00,Payback
48,BBID_2041004,1977-06-22,NaN,MADHYA PRADESH,452001.0,2016-02-28,2655,BB-INDORE-MALHAR MEGA MALL,16,15272,NONPROMO,NaN,108020344,BOURNVITA HEALTH DRINK PP 500g,200.00,Payback
49,BBID_2041004,1977-06-22,NaN,MADHYA PRADESH,452001.0,2016-11-05,2906,BB-INDORE-TREASURE ISLAND,4,33543,NONPROMO,NaN,300091863,PH GROUNDNUT 500g,97.00,Payback
51,BBID_2041049,1972-12-30,male,MADHYA PRADESH,452010.0,2016-06-25,2906,BB-INDORE-TREASURE ISLAND,14,2927,0001150106,AZ 60% OFF+VAT MENS FASHION EOSS LIQ EXC,1000443619003,"KHSH-4166-270-A-HS-CX, 42-SF, RED",461.58,Payback


In [60]:
#discount used how to convert into numeric
#following type of discount has been used
#BBProfitClub,
#Payback
#BBSavingsClub
#T24Club
#FGShoppingFest
#FuturePrivilegeClub

cleaned = customer_full_df.set_index('customerID').discountUsed.str.split(',', expand=True).stack()

In [61]:
cleaned

customerID       
BBID_2044       0          T24Club
                1          Payback
                0          T24Club
                1          Payback
BBID_20494327   0          T24Club
                1          Payback
BBID_20482      0          Payback
BBID_20498      0          Payback
BBID_20499      0          Payback
BBID_204110     0          Payback
BBID_204149     0          Payback
                0          Payback
                0          Payback
BBID_204186     0          Payback
                0          Payback
BBID_204232     0          Payback
                0          Payback
BBID_204242     0          Payback
BBID_204277     0          Payback
BBID_204295     0          Payback
BBID_204322     0          Payback
BBID_204330     0          T24Club
                1          Payback
BBID_204334     0          Payback
BBID_204360     0          Payback
BBID_204451     0          Payback
                0          Payback
BBID_204455     0          Payback
BB

In [62]:
pd.get_dummies(cleaned, prefix='g').groupby(level=0).sum()

,g_BBProfitClub,g_BBSavingsClub,g_FGShoppingFest,g_FuturePrivilegeClub,g_Payback,g_T24Club
customerID,,,,,,
BBID_204100019,0,0,1,0,1,0
BBID_204100060,0,0,1,0,0,0
BBID_204100090,0,0,1,0,0,0
BBID_204100102,0,0,2,0,0,0
BBID_20410014,0,0,0,0,2,1
BBID_204100149,0,0,2,0,0,0
BBID_204100150,0,0,1,0,1,1
BBID_204100220,0,0,1,0,0,0
BBID_204100277,0,0,1,0,0,0


many customer of prod.csv is not present in tender.csv so now to decide the merging ,before merging need to do some kind of feature engineering ,to merge the data 

merging will on the same (customerID,DOB,gender,state,pinCode,TransactionDate,StoreCode,StoreDes,TillNo,TransactionBytill) and we will perform the full join as we dont want to loose the data from either of the data frane 

Create a new Data Customer_Df which includes all the info 


In [ ]:
#create the new data Frame 
#just to check how many are common value
customer_inner_df = pd.merge(tend_df,prod_df,left_on=['customerID','DOB','Gender','State','PinCode','transactionDate','store_code','store_description','till_no','transaction_number_by_till'], right_on = ['customerID','DOB','Gender','State','PinCode','transactionDate','store_code','store_description','till_no','transaction_number_by_till'])


In [ ]:
customer_inner_df

In [ ]:
customer_inner_df.shape

if we see above no of rows is 14466 which is more than prod_df rows which means tend_df is having multiple mode of payment for the same transaction 

In [58]:
#full join to get the both types of data 
customer_full_df = pd.merge(tend_df,prod_df,how='outer',left_on=['customerID','DOB','Gender','State','PinCode','transactionDate','store_code','store_description','till_no','transaction_number_by_till'], right_on = ['customerID','DOB','Gender','State','PinCode','transactionDate','store_code','store_description','till_no','transaction_number_by_till'])


In [41]:
customer_full_df.head(12)

,customerID,DOB,Gender,State,PinCode,transactionDate,store_code,store_description,till_no,tender_type,transaction_number_by_till,payment_amount_by_tender,PaymentUsed,promo_code,promotion_description,product_code,product_description,sale_price_after_promo,discountUsed
0,BBID_2044,1984-04-02,male,NaN,626125.0,2015-09-03,4986,BB-MADURAI-KOCHADAI VILLAGE,22,GVOWN,9208,1000.00,GiftVoucher,NaN,NaN,NaN,NaN,NaN,NaN
1,BBID_2044,1984-04-02,male,NaN,626125.0,2015-09-03,4986,BB-MADURAI-KOCHADAI VILLAGE,22,CASH,9208,10.46,CASH,NaN,NaN,NaN,NaN,NaN,NaN
2,BBID_2044,1984-04-02,male,NaN,626125.0,2016-04-17,4986,BB-MADURAI-KOCHADAI VILLAGE,19,EDCAXIS,90269,677.28,AXIS,NaN,NaN,NaN,NaN,NaN,NaN
3,BBID_2044,1984-04-02,male,NaN,626125.0,2016-04-17,4986,BB-MADURAI-KOCHADAI VILLAGE,19,MOBI,90269,1000.00,One MobiKwik Systems,NaN,NaN,NaN,NaN,NaN,NaN
4,BBID_2044,1984-04-02,male,NaN,626125.0,2016-04-17,4986,BB-MADURAI-KOCHADAI VILLAGE,19,MOBI,90268,1042.00,One MobiKwik Systems,NaN,NaN,NaN,NaN,NaN,NaN
5,BBID_2044,1984-04-02,male,NaN,626125.0,2015-08-23,4986,BB-MADURAI-KOCHADAI VILLAGE,13,GVOWN,76316,1000.00,GiftVoucher,NaN,NaN,NaN,NaN,NaN,NaN
6,BBID_2044,1984-04-02,male,NaN,626125.0,2015-08-23,4986,BB-MADURAI-KOCHADAI VILLAGE,13,MOBI,76316,3490.25,One MobiKwik Systems,NaN,NaN,NaN,NaN,NaN,NaN
7,BBID_2044,1984-04-02,male,NaN,626125.0,2015-09-14,4986,BB-MADURAI-KOCHADAI VILLAGE,8,MOBI,33189,672.00,One MobiKwik Systems,NaN,NaN,NaN,NaN,NaN,NaN
8,BBID_2044,1984-04-02,male,NaN,626125.0,2015-12-13,4986,BB-MADURAI-KOCHADAI VILLAGE,13,GVOWN,16808,500.00,GiftVoucher,NONPROMO,NaN,108020516.0,BRITANNIA BISC GOODAY BUTTER 150g,40.0,"T24Club,Payback"
9,BBID_2044,1984-04-02,male,NaN,626125.0,2015-12-13,4986,BB-MADURAI-KOCHADAI VILLAGE,13,EDCSBI,16808,1196.24,SBI CREDIT CARD,NONPROMO,NaN,108020516.0,BRITANNIA BISC GOODAY BUTTER 150g,40.0,"T24Club,Payback"


In [42]:
customer_full_df.shape

(151128, 19)

In [43]:
#add age as the column which we can get it from the DOB mention before that we will confirm if any null is value is present in the data or not
customer_full_df.isnull().sum()

customerID                         0
DOB                             4796
Gender                          7225
State                          12506
PinCode                         5989
transactionDate                    0
store_code                         0
store_description                  0
till_no                            0
tender_type                     1139
transaction_number_by_till         0
payment_amount_by_tender        1139
PaymentUsed                     2364
promo_code                    135523
promotion_description         146627
product_code                  135523
product_description           135523
sale_price_after_promo        135523
discountUsed                  135523
dtype: int64

#Draw the missing value 

In [44]:
customer_full_df.dtypes

customerID                     object
DOB                            object
Gender                         object
State                          object
PinCode                       float64
transactionDate                object
store_code                      int64
store_description              object
till_no                         int64
tender_type                    object
transaction_number_by_till      int64
payment_amount_by_tender      float64
PaymentUsed                    object
promo_code                     object
promotion_description          object
product_code                  float64
product_description            object
sale_price_after_promo        float64
discountUsed                   object
dtype: object

In [ ]:
#visulaize the same using the missingno 
import missingno as msno
msno.bar(customer_full_df,color='blue')


In [45]:
 customer_full_df['DOB'].head()

0    1984-04-02
1    1984-04-02
2    1984-04-02
3    1984-04-02
4    1984-04-02
Name: DOB, dtype: object

In [14]:
#will try to create new column Age which will be derived from DOB and then will fill the missing value 
import datetime as DT
now=pd.Timestamp(DT.datetime.now())

#customer_full_df['DOB'] = np.where(pd.isnull(customer_full_df['DOB'])==False , pd.to_datetime(customer_full_df['DOB']),' ')



In [15]:
 customer_full_df['DOB']=customer_full_df['DOB'].apply(lambda x: pd.to_datetime(x, errors='coerce', format='%Y/%m/%d'))



In [16]:
def calculateAge(dob):
    currDate =pd.Timestamp(DT.datetime.now())
    age =currDate.year - dob.year - ((currDate.month, currDate.day) < (dob.month, dob.day)) #condition will return 0 or if current month is less than current  year 
#    print age
    return age

In [21]:
#create new column of age 
customer_full_df['Age'] = customer_full_df['DOB'].apply(lGenderambda x:calculateAge(x))

In [22]:
customer_full_df['Age'].head()

0    33.0
1    33.0
2    33.0
3    33.0
4    33.0
Name: Age, dtype: float64

In [23]:
#here to get to know the behaviour of the customer i feel only Age, Gender,discountUsed ,tender_type/payment used,  payment_amount_by_tender(atlast i will try ) 

#before applying anything data should be present in numerical form

#Gender,discountUsed,Tender_Type need to change 

customer_full_df['Gender'] = pd.get_dummies( customer_full_df['Gender'])

In [26]:
customer_full_df['Gender'].unique()

array([0, 1], dtype=uint64)

In [55]:
#now tender type 
#customer_full_df['tender_type'] = pd.Categorical(customer_full_df['tender_type']).codes # Replace strings with an integer

#use label Encoder
from sklearn.preprocessing import LabelEncoder
le_Encoder = LabelEncoder()
customer_full_df['tender_type'] = le_Encoder.fit_transform(customer_full_df['tender_type'])
    

C:\Users\Ravi\Anaconda2\lib\site-packages\numpy\lib\arraysetops.py:216: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))


In [56]:
customer_full_df['tender_type'].head(10)

0    30
1     7
2    19
3    35
4    35
5    30
6    35
7    35
8    30
9    22
Name: tender_type, dtype: int64

In [57]:
customer_full_df['tender_type'].unique()

array([30,  7, 19, 35, 22, 24, 21, 23, 41, 28, 14, 20, 39, 44, 37, 42, 16,
       15,  1, 31, 34,  8, 38,  5, 18,  4, 12, 10, 33, 13, 43, 36, 32,  3,
       27,  2, 40, 29, 11,  6, 26, 17, 25,  9, 45,  0], dtype=int64)

In [ ]:
#get the summary of integer data type and object data type column 
prod_df.describe()

In [ ]:
#for object data type
prod_df.describe(include=[np.object])

In [ ]:
#check for tender also 
tend_df.describe()


In [ ]:
tend_df.describe(include=[np.object])

observation from the data:
    #pin coe transaction code ,store code all are like unique code may not get much info from the summary
    #from summary we can see the top occurence customer ID and their respective details 
    #same for tender data 
    

In [ ]:
###visulaztion to check the pattern ,spread and outlier of data 


In [ ]:
#missing value fill 
prod_df['promotion_description'].fillna('no_promo', inplace=True)
prod_df['Gender'].fillna('no_gender', inplace=True)
prod_df['State'].fillna('no_state', inplace=True)
prod_df['PinCode'].fillna(-1, inplace=True)
prod_df['DOB'].fillna("1", inplace=True)

In [ ]:


## convert data into numeric / float

for c in prod_df.columns:
    lbl = LabelEncoder()
    if prod_df[c].dtype == 'object' and c not in ['store_description','customerID','transactionDate']:
        prod_df[c] = lbl.fit_transform(prod_df[c])



In [ ]:
prod_df.head(5)

In [ ]:
stores = list(set(prod_df['store_code']))

In [ ]:
stores

In [ ]:
cluster_labels = []
cluster_store = []
cluster_data = []
cluster_customers = []
cluster_score = []

In [ ]:

for x in stores:
    cld = prod_df[prod_df['store_code'] == x]
    cluster_customers.append(cld['customerID'])
    cld.drop(['store_code','customerID','transactionDate','store_description'], axis=1, inplace=True)
    
    rbs = RobustScaler()
    cld2 = rbs.fit_transform(cld)
    
    km1 = KMeans(n_clusters=3)
    km2 = km1.fit(cld2)
    label = km2.predict(cld2)
    
    s_score = silhouette_score(cld2, label)
    cluster_score.append(s_score)
    
    cluster_labels.append(label)
    cluster_store.append(np.repeat(x, cld.shape[0]))
    cluster_data.append(cld2)

In [ ]:
np.mean(cluster_score)

In [ ]:
## merge list into ndarray
cluster_data = np.concatenate(cluster_data)

In [ ]:
## check if the array has same rows as products file - Yes!
cluster_data.shape

In [ ]:
## convert nested lists as 1d array
cluster_customers = np.concatenate(cluster_customers)
cluster_store = np.concatenate(cluster_store)
cluster_labels = np.concatenate(cluster_labels)

In [ ]:


## create submission files
sub1 = pd.DataFrame({'customerID':cluster_customers, 'store_code':cluster_store, 'cluster':cluster_labels})



In [ ]:
np.savetxt('result/subOne_18.txt', cluster_data)
sub1.to_csv('result/subtwo_18.csv', index=False)

In [ ]:
#check the spread of the data 
# using sea born library check the spread of data and relation between them 
#importing seaborn for statistical plots
import seaborn as sns


In [ ]:
#heat map for correlation 
#calculate the correlation matrix
corr = prod_df.corr()
cmap = cmap=sns.diverging_palette(5, 250, as_cmap=True)


#draw the correlation table 
def magnify():
    return [dict(selector="th",
                 props=[("font-size", "7pt")]),
            dict(selector="td",
                 props=[('padding', "0em 0em")]),
            dict(selector="th:hover",
                 props=[("font-size", "12pt")]),
            dict(selector="tr:hover td:hover",
                 props=[('max-width', '200px'),
                        ('font-size', '12pt')])
]

corr.style.background_gradient(cmap, axis=1)\
    .set_properties(**{'max-width': '80px', 'font-size': '10pt'})\
    .set_caption("Hover to magify")\
    .set_precision(2)\
    .set_table_styles(magnify())

looks like no correlation here :( 